# Topological Sorting

### Topological sorting for Directed Acyclic Graph `  (DAG)`   is a linear ordering of vertices such that for every directed edge u-v, vertex u comes before v in the ordering.

## Topological Sort: Queue vs. Stack

### 1. Using a Queue (Preferred)
- **Queue** follows **FIFO** and processes nodes in the correct topological order.
- Nodes with in-degree 0 are enqueued, and as nodes are processed, their neighbors’ in-degrees are updated.

### 2. Using a Stack
- **Stack** follows **LIFO**, so it processes nodes in reverse topological order.
- After processing, the stack must be **reversed** to get the correct order.



1. **What is the time complexity of this topological sort algorithm?**
   - **Answer**: **O(V + E)**: 
     - **O(V)** for calculating in-degrees.
     - **O(E)** for iterating over edges to update in-degrees.

2. **What would happen if the graph contains a cycle?**
   - **Answer**: The algorithm detects the cycle by checking `count != V`. If true, topological sort is not possible.

3. **Can this algorithm work with a cyclic graph?**
   - **Answer**: No, it assumes a **DAG** (Directed Acyclic Graph). If a cycle exists, it won't return a valid topological order.



In [31]:
from collections import defaultdict, deque

class Graph:
    def __init__(self, vertices):
        self.graph = defaultdict(list)  # adjacency list
        self.V = vertices               # number of vertices

    def add_edge(self, u, v):
        self.graph[u].append(v)        # edge from node u to node v

    def topological_sort(self):
        # Step 1: Calculate in-degrees of all nodes
        in_degree = [0] * self.V
        for u in self.graph:
            for v in self.graph[u]:
                in_degree[v] += 1

        print("Initial in-degrees:", in_degree)  

        # Step 2: Enqueue nodes with 0 in-degree
        queue = deque()
        for i in range(self.V):
            if in_degree[i] == 0:
                queue.append(i) #appened all zero degree nodes.

        print("Starting nodes (0 in-degree):", list(queue))  # tracing

        # Step 3: Process the graph
        topo_order = []
        #count is a variable that is incremented each time a node is
        # removed from the queue and added to the topological order.
        #It keeps track of how many nodes we have successfully sorted.
        count = 0

        while queue:
            u = queue.popleft()
            topo_order.append(u)
            print(f"Node {u} added to topological order.")  # tracing

            for v in self.graph[u]:
                in_degree[v] -= 1
                print(f"Decreased in-degree of node {v} to {in_degree[v]}.")  # tracing
                if in_degree[v] == 0:
                    queue.append(v)
                    print(f"Node {v} added to queue.")  # tracing

            count += 1
#if there's a cycle, at least one node will always have a non-zero in-degree,
# because it's waiting for another node that depends on it — forming a loop!
        if count != self.V:
            print("Cycle detected! No topological ordering possible.")
        else:
            print("Topological Sort Result:", topo_order)

        return topo_order


# Create graph instance
g = Graph(6)
g.add_edge(5, 2)
g.add_edge(5, 0)
g.add_edge(4, 0)
g.add_edge(4, 1)
g.add_edge(2, 3)
g.add_edge(3, 1)

# Perform topological sort
topo = g.topological_sort()


Initial in-degrees: [2, 2, 1, 1, 0, 0]
Starting nodes (0 in-degree): [4, 5]
Node 4 added to topological order.
Decreased in-degree of node 0 to 1.
Decreased in-degree of node 1 to 1.
Node 5 added to topological order.
Decreased in-degree of node 2 to 0.
Node 2 added to queue.
Decreased in-degree of node 0 to 0.
Node 0 added to queue.
Node 2 added to topological order.
Decreased in-degree of node 3 to 0.
Node 3 added to queue.
Node 0 added to topological order.
Node 3 added to topological order.
Decreased in-degree of node 1 to 0.
Node 1 added to queue.
Node 1 added to topological order.
Topological Sort Result: [4, 5, 2, 0, 3, 1]


# Prim’s Algorithm for Minimum Spanning Tree (MST)

### Time Complexity -> `O((E+V)*log(V))`

###  Time Complexity Breakdown

 **Total cost of pops** → `O(V log V)`  
- We pop from the heap once for each node (at most `V` times).  
- Each `pop` takes `O(log V)` time (because the heap has up to `V` elements).  

 **Total cost of pushes** → `O(E log V)`  
- We push each edge into the heap (up to `E` times).  
- Each `push` takes `O(log V)` time.

 **Total time complexity**  
\[
O(V \log V) + O(E \log V) = O((V + E) \log V)
\]

**Total= O(VlogV)+O(ElogV)=`O((V+E)logV)`**




###  Disadvantages

1. **Performance:** Can be slow on dense graphs with many edges, as it requires iterating over all edges at least once.  
2. **Memory usage:** Relies on a priority queue, which can take up extra memory and slow down the algorithm on very large graphs.  
3. **Start node sensitivity:** The choice of starting node can affect the MST output.


In [ ]:
import heapq
from collections import defaultdict

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)  # adjacency list: u -> [(v, weight)]
#why defaultdict?
#If you try to access or modify a key that doesn't exist in the dictionary, 
# it will automatically create a new entry for that key with a default value.
   
    def add_edge(self, u, v, weight):
        # Since the graph is undirected, add both u->v and v->u
        #no need for graph[0] = [] as it will be created automatically
        #as it adds empty list even if the key doesn't exist.
        # This is a tuple (v, weight) 
        #immutable and ordered
        #Ordering: Ensures that each element in the tuple has a specific place 
        #first for the node, second for the weight

        self.graph[u].append((v, weight))
        self.graph[v].append((u, weight))

    def prim_mst(self, start):
        visited = set()  #prevent revisiting nodes
        min_heap = [(0, start, -1)]  # (weight, current_vertex, parent_vertex)
        mst_edges = []
        total_weight = 0

        print("Starting Prim's Algorithm from node:", start)

        while min_heap:
            weight, u, parent = heapq.heappop(min_heap)

            if u in visited:
                continue

            visited.add(u)
            total_weight += weight

            if parent != -1:
                mst_edges.append((parent, u, weight))
                print(f"Added edge: ({parent} - {u}, weight: {weight})")

            for v, w in self.graph[u]:
                if v not in visited:
                    heapq.heappush(min_heap, (w, v, u))
                    print(f"Consider edge: ({u} - {v}, weight: {w})")

        print("\nTotal weight of MST:", total_weight)
        return mst_edges, total_weight
g = Graph()
g.add_edge(0, 1, 4)
g.add_edge(0, 2, 3)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 2)
g.add_edge(2, 3, 4)
g.add_edge(3, 4, 2)
g.add_edge(4, 5, 6)

g.prim_mst(0)

#done tracing it




Starting Prim's Algorithm from node: 0
Consider edge: (0 - 1, weight: 4)
Consider edge: (0 - 2, weight: 3)
Added edge: (0 - 2, weight: 3)
Consider edge: (2 - 1, weight: 1)
Consider edge: (2 - 3, weight: 4)
Added edge: (2 - 1, weight: 1)
Consider edge: (1 - 3, weight: 2)
Added edge: (1 - 3, weight: 2)
Consider edge: (3 - 4, weight: 2)
Added edge: (3 - 4, weight: 2)
Consider edge: (4 - 5, weight: 6)
Added edge: (4 - 5, weight: 6)

Total weight of MST: 14


([(0, 2, 3), (2, 1, 1), (1, 3, 2), (3, 4, 2), (4, 5, 6)], 14)